In [59]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
df = pd.read_csv('G:/New 20121 Dekstop Files/thesis 2021/employee attrition dataset/employee_attrition.csv')

# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

In [3]:
df.sample(6)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
23,21,No,Travel_Rarely,391,Research & Development,15,2,Life Sciences,1,30,...,4,80,0,0,6,3,0,0,0,0
47,37,No,Travel_Rarely,408,Research & Development,19,2,Life Sciences,1,61,...,1,80,0,8,1,3,1,0,0,0
915,21,Yes,Travel_Frequently,251,Research & Development,10,2,Life Sciences,1,1279,...,3,80,0,2,2,1,2,2,2,2
1358,31,No,Travel_Rarely,1079,Sales,10,2,Medical,1,1912,...,4,80,1,8,2,3,5,2,1,4
1211,37,No,Travel_Frequently,1278,Sales,1,4,Medical,1,1700,...,3,80,2,6,2,2,6,3,1,3
992,39,No,Non-Travel,1485,Research & Development,25,2,Life Sciences,1,1397,...,2,80,1,13,2,3,6,4,0,5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [5]:
df['Attrition'].replace({'Yes':1,'No':0}, inplace = True)

In [6]:
df.Attrition.unique()

array([1, 0], dtype=int64)

In [7]:
df.Attrition.value_counts()

0    1233
1     237
Name: Attrition, dtype: int64

#  Data Cleaning 

In [8]:
for col in df:
    print(f'{col}: {df[col].unique()}') 

Age: [41 49 37 33 27 32 59 30 38 36 35 29 31 34 28 22 53 24 21 42 44 46 39 43
 50 26 48 55 45 56 23 51 40 54 58 20 25 19 57 52 47 18 60]
Attrition: [1 0]
BusinessTravel: ['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']
DailyRate: [1102  279 1373 1392  591 1005 1324 1358  216 1299  809  153  670 1346
  103 1389  334 1123 1219  371  673 1218  419  391  699 1282 1125  691
  477  705  924 1459  125  895  813 1273  869  890  852 1141  464 1240
 1357  994  721 1360 1065  408 1211 1229  626 1434 1488 1097 1443  515
  853 1142  655 1115  427  653  989 1435 1223  836 1195 1339  664  318
 1225 1328 1082  548  132  746  776  193  397  945 1214  111  573 1153
 1400  541  432  288  669  530  632 1334  638 1093 1217 1353  120  682
  489  807  827  871  665 1040 1420  240 1280  534 1456  658  142 1127
 1031 1189 1354 1467  922  394 1312  750  441  684  249  841  147  528
  594  470  957  542  802 1355 1150 1329  959 1033 1316  364  438  689
  201 1427  857  933 1181 1395  662 1436  194  967 1496 11

In [9]:
df.describe()

,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,...,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,0.161224,802.485714,9.192517,2.912925,1.0,1024.865306,2.721769,65.891156,2.729932,...,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,0.367863,403.509100,8.106864,1.024165,0.0,602.024335,1.093082,20.329428,0.711561,...,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,0.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,0.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,0.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,66.000000,3.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,0.000000,1157.000000,14.000000,4.000000,1.0,1555.750000,4.000000,83.750000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [10]:
df['Age'] = df['Age']/60
df['DailyRate'] = df['DailyRate']/1499
df['HourlyRate'] = df['HourlyRate']/100

In [11]:
df.drop(['Over18','StandardHours'], axis=1,inplace=True)

# Label Encoding

In [12]:
label_encoder =  preprocessing.LabelEncoder()
df['BusinessTravel'] = label_encoder.fit_transform(df['BusinessTravel'])
df['Department'] = label_encoder.fit_transform(df['Department'])
df['EducationField'] = label_encoder.fit_transform(df['EducationField'])
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['JobRole'] = label_encoder.fit_transform(df['JobRole'])
df['MaritalStatus'] = label_encoder.fit_transform(df['MaritalStatus'])
# df['Over18'] = label_encoder.fit_transform(df['Over18'])
df['OverTime'] = label_encoder.fit_transform(df['OverTime'])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1470 non-null   float64
 1   Attrition                 1470 non-null   int64  
 2   BusinessTravel            1470 non-null   int32  
 3   DailyRate                 1470 non-null   float64
 4   Department                1470 non-null   int32  
 5   DistanceFromHome          1470 non-null   int64  
 6   Education                 1470 non-null   int64  
 7   EducationField            1470 non-null   int32  
 8   EmployeeCount             1470 non-null   int64  
 9   EmployeeNumber            1470 non-null   int64  
 10  EnvironmentSatisfaction   1470 non-null   int64  
 11  Gender                    1470 non-null   int32  
 12  HourlyRate                1470 non-null   float64
 13  JobInvolvement            1470 non-null   int64  
 14  JobLevel

In [14]:
df.sample(3)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1263,0.700000,0,2,0.570380,1,12,3,3,1,1768,...,4,2,3,7,6,2,5,3,0,4
523,0.833333,0,2,0.805203,1,28,1,3,1,716,...,3,3,3,20,3,3,20,8,3,8
669,0.650000,1,2,0.748499,1,6,3,3,1,932,...,4,4,0,8,2,1,2,2,2,2


In [15]:
df1 = df.copy()

In [16]:
df1

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.683333,1,2,0.735157,2,1,2,1,1,1,...,3,1,0,8,0,1,6,4,0,5
1,0.816667,0,1,0.186124,1,8,1,1,1,2,...,4,4,1,10,3,3,10,7,1,7
2,0.616667,1,2,0.915944,1,2,2,4,1,4,...,3,2,0,7,3,3,0,0,0,0
3,0.550000,0,1,0.928619,1,3,4,1,1,5,...,3,3,0,8,3,3,8,7,3,0
4,0.450000,0,2,0.394263,1,2,1,3,1,7,...,3,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,0.600000,0,1,0.589726,1,23,2,3,1,2061,...,3,3,1,17,3,3,5,2,0,3
1466,0.650000,0,2,0.408939,1,6,1,3,1,2062,...,3,1,1,9,5,3,7,7,1,7
1467,0.450000,0,2,0.103402,1,4,3,1,1,2064,...,4,2,1,6,0,3,6,2,0,3
1468,0.816667,0,1,0.682455,2,2,3,3,1,2065,...,3,4,0,17,3,2,9,6,0,8


In [17]:
df1_class0 = df1[df1.Attrition==0]
df1_class1 = df1[df1.Attrition==1]

In [18]:
df1_class0.shape

(1233, 33)

In [19]:
df1_class1.shape

(237, 33)

In [20]:
subset1 = pd.concat([df1_class1[0:75],df1_class0[0:75]], axis=0)

x = subset1.drop('Attrition', axis='columns')
y = subset1.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

# Random Forest
rnf = RandomForestClassifier(n_estimators=30, random_state=51)
rnf.fit(xtrain,ytrain)
# Accuracy on Training 
predictions_train = rnf.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = rnf.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  1.0

Testing Accuracy :  0.8666666666666667


In [21]:
subset2 = pd.concat([df1_class1[76:151],df1_class0[76:151]], axis=0)

x = subset2.drop('Attrition', axis='columns')
y = subset2.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

# SVM
svm_model = SVC(gamma = 'auto')
svm_model.fit(xtrain,ytrain)
# Accuracy on Training 
predictions_train = svm_model.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = svm_model.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  1.0

Testing Accuracy :  0.5


In [22]:
subset3 = pd.concat([df1_class1[152:236],df1_class0[152:236]], axis=0)

x = subset3.drop('Attrition', axis='columns')
y = subset3.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 
# XGBoost
classifier=XGBClassifier()
classifier.fit(xtrain,ytrain)
predictions_train =classifier.predict(xtrain)
acc=accuracy_score(ytrain, predictions_train)
print('\nTraining Accuracy :', acc)
predictions_test = classifier.predict(xtest)
accc=accuracy_score(ytest, predictions_test)
print('\nTesting Accuracy :', accc)

F:\New_Anaconda_Inst_Loc\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:04:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

Training Accuracy : 1.0

Testing Accuracy : 1.0


In [23]:
subset4 = pd.concat([df1_class1[0:75],df1_class0[237:312]], axis=0)

x = subset4.drop('Attrition', axis='columns')
y = subset4.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

# Logistic Regression
lrmodel = LogisticRegression()
lrmodel.fit(xtrain,ytrain)
# Accuracy on Training 
predictions_train = lrmodel.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = lrmodel.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  0.7583333333333333

Testing Accuracy :  0.6333333333333333


F:\New_Anaconda_Inst_Loc\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
subset5 = pd.concat([df1_class1[76:151],df1_class0[313:388]], axis=0)

x = subset5.drop('Attrition', axis='columns')
y = subset5.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

# Light GBM
model = lgb.LGBMClassifier(learning_rate=0.09,max_depth=-5,random_state=42)
model.fit(xtrain,ytrain,eval_set=[(xtest,ytest),(xtrain,ytrain)],
          verbose=20,eval_metric='logloss')
# Accuracy on Training 
predictions_train = model.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = model.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))

F:\New_Anaconda_Inst_Loc\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[20]	training's binary_logloss: 0.0837564	valid_0's binary_logloss: 0.0837564
[40]	training's binary_logloss: 0.0133266	valid_0's binary_logloss: 0.0133266
[60]	training's binary_logloss: 0.00218954	valid_0's binary_logloss: 0.00218954
[80]	training's binary_logloss: 0.000361568	valid_0's binary_logloss: 0.000361568
[100]	training's binary_logloss: 5.97569e-05	valid_0's binary_logloss: 5.97569e-05

Training Accuracy :  1.0

Testing Accuracy :  1.0


In [25]:
subset6 = pd.concat([df1_class1[152:237],df1_class0[389:474]], axis=0)

x = subset6.drop('Attrition', axis='columns')
y = subset6.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

# CatBoost
catmodel = CatBoostClassifier(iterations=100)
catmodel.fit(xtrain,ytrain,plot=False)

# Accuracy on Training 
predictions_train = catmodel.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = catmodel.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))

Learning rate set to 0.036304
0:	learn: 0.6421485	total: 300ms	remaining: 29.7s
1:	learn: 0.6013609	total: 306ms	remaining: 15s
2:	learn: 0.5481563	total: 311ms	remaining: 10.1s
3:	learn: 0.5053061	total: 316ms	remaining: 7.59s
4:	learn: 0.4718101	total: 321ms	remaining: 6.09s
5:	learn: 0.4401958	total: 324ms	remaining: 5.08s
6:	learn: 0.4144969	total: 327ms	remaining: 4.35s
7:	learn: 0.3854309	total: 330ms	remaining: 3.8s
8:	learn: 0.3602492	total: 334ms	remaining: 3.37s
9:	learn: 0.3415879	total: 337ms	remaining: 3.03s
10:	learn: 0.3253246	total: 340ms	remaining: 2.75s
11:	learn: 0.3005120	total: 344ms	remaining: 2.52s
12:	learn: 0.2819271	total: 347ms	remaining: 2.32s
13:	learn: 0.2641211	total: 349ms	remaining: 2.14s
14:	learn: 0.2464914	total: 352ms	remaining: 1.99s
15:	learn: 0.2347550	total: 354ms	remaining: 1.86s
16:	learn: 0.2183808	total: 356ms	remaining: 1.74s
17:	learn: 0.2086025	total: 359ms	remaining: 1.63s
18:	learn: 0.1941349	total: 361ms	remaining: 1.54s
19:	learn: 0.1

In [26]:
subset7 = pd.concat([df1_class1[0:75],df1_class0[475:550]], axis=0)

x = subset7.drop('Attrition', axis='columns')
y = subset7.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

#MultinomialNB
nbmodel = MultinomialNB()
nbmodel.fit(xtrain,ytrain)

# Accuracy on Training 
predictions_train = nbmodel.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = nbmodel.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  0.5916666666666667

Testing Accuracy :  0.6666666666666666


In [27]:
subset8 = pd.concat([df1_class1[76:151],df1_class0[551:625]], axis=0)

x = subset8.drop('Attrition', axis='columns')
y = subset8.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

# Adaboost
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1)
model = abc.fit(xtrain, ytrain)

# Accuracy on Training 
predictions_train = model.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = model.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  1.0

Testing Accuracy :  0.9


In [28]:
subset9 = pd.concat([df1_class1[152:237],df1_class0[626:711]], axis=0)

x = subset9.drop('Attrition', axis='columns')
y = subset9.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

# Stochastic Gradient Boosting
model = GradientBoostingClassifier()
model.fit(xtrain, ytrain)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model on the dataset
n_scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# # Accuracy on Training 
# predictions_train = model.predict(xtrain)
# print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# # Accuracy on Testing
# predictions_test = model.predict(xtest)
# print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))

Mean Accuracy: 1.000 (0.000)


In [38]:
subset10 = pd.concat([df1_class1[0:75],df1_class0[712:787]], axis=0)

x = subset10.drop('Attrition', axis='columns')
y = subset10.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

# K Neighbors Classifier
classifier= KNeighborsClassifier(n_neighbors=10, metric='minkowski', p=2 )  
classifier.fit(xtrain, ytrain)  

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# evaluate the model on the dataset
n_scores = cross_val_score(classifier, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# Accuracy on Training 
predictions_train = classifier.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = classifier.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  0.6916666666666667

Testing Accuracy :  0.7333333333333333


In [41]:
subset11 = pd.concat([df1_class1[76:151],df1_class0[788:863]], axis=0)
x = subset11.drop('Attrition', axis='columns')
y = subset11.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

# Decision Tree Classifier
dt = DecisionTreeClassifier()
dt.fit(xtrain,ytrain)

# # define the evaluation method
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# # evaluate the model on the dataset
# n_scores = cross_val_score(classifier, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
# # report performance
# print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# Accuracy on Training 
predictions_train = dt.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = dt.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  1.0

Testing Accuracy :  1.0


In [53]:
subset12 = pd.concat([df1_class1[152:237],df1_class0[864:949]], axis=0)
x = subset12.drop('Attrition', axis='columns')
y = subset12.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

# Linear Discriminant Analysis
model = LinearDiscriminantAnalysis()
model.fit(xtrain,ytrain)

# # define the evaluation method
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# # evaluate the model on the dataset
# n_scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
# # report performance
# print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# Accuracy on Training 
predictions_train = model.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = model.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  0.8235294117647058

Testing Accuracy :  0.8529411764705882


In [58]:
subset13 = pd.concat([df1_class1[0:75],df1_class0[950:1025]], axis=0)
x = subset13.drop('Attrition', axis='columns')
y = subset13.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

# Gaussian Naive Bayes
classifier = GaussianNB()
classifier.fit(xtrain, ytrain)

# # define the evaluation method
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# # evaluate the model on the dataset
# n_scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
# # report performance
# print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# Accuracy on Training 
predictions_train = classifier.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = classifier.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  1.0

Testing Accuracy :  1.0


In [61]:
subset14 = pd.concat([df1_class1[76:151],df1_class0[1026:1117]], axis=0)
x = subset14.drop('Attrition', axis='columns')
y = subset14.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

# Bernoulli Naive Bayes
model = BernoulliNB()
model.fit(xtrain, ytrain)

# # define the evaluation method
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# # evaluate the model on the dataset
# n_scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
# # report performance
# print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# Accuracy on Training 
predictions_train = model.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = model.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  0.7651515151515151

Testing Accuracy :  0.7058823529411765


In [68]:
subset15 = pd.concat([df1_class1[152:237],df1_class0[1118:1233]], axis=0)
x = subset15.drop('Attrition', axis='columns')
y = subset15.Attrition

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y ) 

# Random Forest
rnf = RandomForestClassifier(n_estimators=25, random_state=51)
rnf.fit(xtrain,ytrain)
# Accuracy on Training 
predictions_train = rnf.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = rnf.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  1.0

Testing Accuracy :  0.925
